# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo
import json

In [2]:
client = pymongo.MongoClient("mongo:27017") 
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Nettoyer les données

In [4]:
# Convertir colonne date en date
df_youtube['date']= pd.to_datetime("2018." + df_youtube['date'].astype(str), format='%Y.%d.%m')

In [5]:
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,2018-09-13
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,2018-09-13


In [6]:
# Récupérer uniquement la dernière vidéo 
# pour celles présente plusieurs fois dans df_youtube

for i in df_youtube['video_id'].unique():
    s_df= df_youtube[df_youtube['video_id']==i]
    latest = s_df['date'].max()
    oldest = s_df[s_df['date']!=latest]
    df_youtube= df_youtube.drop(oldest.index)

In [7]:
# Changer l'id des categories par leurs noms

f = open("./data/US_category_id.json", "r")
categories = json.load(f)

cat={} # id des categories associés à leurs noms
for i in categories['items']:
    cat[i['id']]= i['snippet']['title']

# modification dans la dataframe
for i in df_youtube['category_id']:
    i= df_youtube['category_id'].replace(i, cat[str(i)], inplace=True)
df_youtube= df_youtube.rename(columns={"category_id":"category"})

In [8]:
df_youtube.head(2)

,video_id,title,channel_title,category,tags,views,likes,dislikes,comment_total,thumbnail_link,date
22,5ywKal6-anc,Gigi Hadid Loses High Heel During Fashion Week...,TMZ,Entertainment,TMZ2016FS11221|TMZ|Hollywood|Celebrity|Enterta...,703750,2921,2196,1042,https://i.ytimg.com/vi/5ywKal6-anc/default.jpg,2018-09-13
33,GGm0FQ6i74U,What Hillary Clinton really thinks,Vox,News & Politics,vox.com|vox|explain|ezra klein|hillary clinton...,244433,9143,6826,4838,https://i.ytimg.com/vi/GGm0FQ6i74U/default.jpg,2018-09-13


In [9]:
# Modifications des tags afin de les transformer en liste
tags= []
for i in df_youtube['tags'] :
    tags.append(i.split("|"))
# Remplace l'ancienne colonne tag par la nouvelle
df_youtube.drop(columns= 'tags')
df_youtube['tags']= tags

In [10]:
df_youtube.head(2)

,video_id,title,channel_title,category,tags,views,likes,dislikes,comment_total,thumbnail_link,date
22,5ywKal6-anc,Gigi Hadid Loses High Heel During Fashion Week...,TMZ,Entertainment,"[TMZ2016FS11221, TMZ, Hollywood, Celebrity, En...",703750,2921,2196,1042,https://i.ytimg.com/vi/5ywKal6-anc/default.jpg,2018-09-13
33,GGm0FQ6i74U,What Hillary Clinton really thinks,Vox,News & Politics,"[vox.com, vox, explain, ezra klein, hillary cl...",244433,9143,6826,4838,https://i.ytimg.com/vi/GGm0FQ6i74U/default.jpg,2018-09-13


### Importer les données

In [11]:
df_youtube= df_youtube.rename(columns={"video_id":"_id"})

# Spécifier ID des documents

In [12]:
payload = json.loads(df_youtube.to_json(orient='records'))
collection.delete_many({}) #pour nettoyer la collection
collection.insert_many(payload)

In [13]:
collection.find_one() # Vérifie que les modifications ont bien été réalisé

{'_id': '5ywKal6-anc',
 'title': 'Gigi Hadid Loses High Heel During Fashion Week, Walks It Off Like a Pro | TMZ',
 'channel_title': 'TMZ',
 'category': 'Entertainment',
 'tags': ['TMZ2016FS11221',
  'TMZ',
  'Hollywood',
  'Celebrity',
  'Entertainment',
  'Famous',
  'Hollywood News',
  'Fame',
  'Entertainment News'],
 'views': 703750,
 'likes': 2921,
 'dislikes': 2196,
 'comment_total': 1042,
 'thumbnail_link': 'https://i.ytimg.com/vi/5ywKal6-anc/default.jpg',
 'date': 1536796800000}

## Question 1  

In [14]:
# vidéos de la chaîne Apple
cur= collection.find({"channel_title":"Apple"})
list(cur)

[{'_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category': 'Science & Technology',
  'tags': ['Apple',
   'iPhone 10',
   'iPhone Ten',
   'iPhone',
   'Portrait Lighting',
   'A11 Bionic',
   'augmented reality',
   'emoji',
   'animoji',
   'Face ID',
   'Apple Pay',
   'camera',
   'smartphone'],
  'views': 19707391,
  'likes': 381919,
  'dislikes': 69465,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 1537142400000},
 {'_id': 'A9k88sMyiJM',
  'title': 'Apple Watch Series 3 + Apple Music — Roll — Apple',
  'channel_title': 'Apple',
  'category': 'Science & Technology',
  'tags': ['Apple',
   'airpods',
   'apple watch',
   'apple watch series 3',
   'apple watch cellular',
   'apple watch 4g',
   'apple music',
   '40 million songs on your wrist',
   'apple skateboard ad',
   'apple train station ad',
   'apple roll ad',
   'apple watch skateboard ad',
   'apple watch 

## Question 2

In [15]:
# nombre de catégories différentes
len(collection.distinct('category'))

16

## Question 3

In [16]:
# Je l'ai directement réalisé dans la partie nettoyage, car plus simple
# Mais si nous devions vraiment modifier et mettre à jour avec une requêtre update
# nous aurions dû utiliser une boucle for afin qu'elle parcourt chacun des documents
# et qu'elle mettre à jour avec les tags un à un

## Question 4

In [17]:
# Les 3 vidéos les plus vues
cur = collection.find().sort([("views",-1)]).limit(3)
list(cur)

[{'_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category': 'Music',
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 1537574400000},
 {'_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'channel_title': 'ZaynVEVO',
  'category': 'Music',
  'tags': ['Dusk Till Dawn', 'Pop', 'RCA Records Label', 'ZAYN feat. Sia'],
  'views': 36323498,
  'likes': 1431683,
  'dislikes': 28049,
  'comment_total': 100661,
  'thumbnail_link': 'https://i.ytimg.com/vi/tt2k8PGm-TI/default.jpg',
  'date': 1536796800000},
 {'_id': 'LunHybOKIjU',
  'title': 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher',
  'channel_title': 'BETNetworks',
  'category': 'Entertainment',
  'tags': ['BET',
   'BET Networks',
   'BET Music',
   'Black Entertainment Telev

## Question 5

In [18]:
# nombre moyen de vues en fonction de la catégorie
cur = collection.aggregate([
    {"$group" : {
        "_id" : "$category", 
        "views_mean" : {"$avg": "$views"}
    }}
])
list(cur)

[{'_id': 'Pets & Animals', 'views_mean': 592803.6052631579},
 {'_id': 'Film & Animation', 'views_mean': 954403.7227722772},
 {'_id': 'Music', 'views_mean': 1384238.0746268656},
 {'_id': 'Gaming', 'views_mean': 772871.5},
 {'_id': 'Sports', 'views_mean': 611369.9463087248},
 {'_id': 'Autos & Vehicles', 'views_mean': 621971.3783783783},
 {'_id': 'People & Blogs', 'views_mean': 878460.2178988326},
 {'_id': 'Travel & Events', 'views_mean': 425081.27777777775},
 {'_id': 'Howto & Style', 'views_mean': 548577.6679389313},
 {'_id': 'Science & Technology', 'views_mean': 850247.4039735099},
 {'_id': 'Comedy', 'views_mean': 1294886.5117370891},
 {'_id': 'Entertainment', 'views_mean': 1049658.8490566039},
 {'_id': 'Nonprofits & Activism', 'views_mean': 681178.2},
 {'_id': 'News & Politics', 'views_mean': 500835.595959596},
 {'_id': 'Shows', 'views_mean': 8726.0},
 {'_id': 'Education', 'views_mean': 542851.9680851063}]

## Question 6 

In [19]:
# chaines Youtube avec la plus grande moyenne de likes
cur = collection.aggregate([
    {"$group" : {
        "_id" : "$channel_title", 
        "views_mean" : {"$avg": "$views"}
    }},
    {"$sort" : {"views_mean": -1}},
    {"$limit": 5}
])
list(cur)

[{'_id': 'ZaynVEVO', 'views_mean': 36323498.0},
 {'_id': 'BETNetworks', 'views_mean': 33191594.0},
 {'_id': 'shakiraVEVO', 'views_mean': 32136948.0},
 {'_id': 'ibighit', 'views_mean': 28268247.0},
 {'_id': 'Primitive Technology', 'views_mean': 15579127.0}]